In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import warnings
warnings.filterwarnings('ignore')

# Tentar diferentes abordagens para carregar o CSV problemático
print("Tentando carregar o dataset...")

# Método 1: Tentar com error_bad_lines para pular linhas problemáticas
try:
    df = pd.read_csv('csv_longo/projetos_resultados_pessoas_valores.csv', 
                     encoding='UTF-8', 
                     low_memory=False,
                     sep=';',  # Garantir que está usando vírgula
                     quotechar='"')  # Tratar aspas
    print("✓ Dados carregados")
except Exception as e:
    print(f"Erro no método 1: {e}")
    
    # Método 2: Ler linha por linha para identificar o problema
    try:
        # Primeiro, descobrir quantas colunas deveria ter
        with open('csv_longo/projetos_resultados_pessoas_valores.csv', 'r', encoding='latin-1') as f:
            header = f.readline()
            n_cols = len(header.split(','))
            print(f"Número esperado de colunas: {n_cols}")
        
        # Carregar pulando linhas ruins
        df = pd.read_csv('csv_longo/projetos_resultados_pessoas_valores.csv',
                        encoding='latin-1',
                        low_memory=False,
                        on_bad_lines='warn',
                        engine='python',
                        sep=',',
                        quotechar='"',
                        escapechar='\\')
        print("✓ Dados carregados com engine='python'")
    except:
        # Método 3: Última tentativa - ler em chunks
        chunks = []
        chunk_size = 10000
        problematic_lines = []
        
        for i, chunk in enumerate(pd.read_csv('csv_longo/projetos_resultados_pessoas_valores.csv',
                                             encoding='latin-1',
                                             low_memory=False,
                                             on_bad_lines='skip',
                                             chunksize=chunk_size)):
            chunks.append(chunk)
            print(f"  Chunk {i+1} carregado ({len(chunk)} linhas)")
            
        df = pd.concat(chunks, ignore_index=True)
        print("✓ Dados carregados em chunks")

print("\n" + "="*80)
print("DADOS CARREGADOS COM SUCESSO")
print("="*80)
print(f"Total de registros: {len(df):,}")
print(f"Total de colunas: {df.shape[1]}")

# ============================================================
# FILTRO CRÍTICO: REMOVER DADOS DE 2023 (DADOS INCOMPLETOS)
# ============================================================
print("\n" + "="*80)
print("APLICANDO FILTRO CRÍTICO - REMOVENDO 2023")
print("="*80)

# Verificar distribuição antes do filtro
if 'ano_referencia' in df.columns:
    print(f"Registros por ano ANTES do filtro:")
    print(df['ano_referencia'].value_counts().sort_index())
    
    # Contagem antes
    total_antes = len(df)
    registros_2023 = (df['ano_referencia'] == 2023).sum()
    
    # APLICAR FILTRO - REMOVER 2023
    df = df[df['ano_referencia'] != 2023].copy()
    
    print(f"\n✓ Removidos {registros_2023:,} registros de 2023 ({registros_2023/total_antes*100:.1f}%)")
    print(f"✓ Dataset filtrado: {len(df):,} registros (2018-2022)")
    
    print(f"\nRegistros por ano APÓS filtro:")
    print(df['ano_referencia'].value_counts().sort_index())
else:
    print("⚠️ Coluna 'ano_referencia' não encontrada - verificar estrutura dos dados")

print("="*80)

# Verificar colunas esperadas
colunas_esperadas = ['id_projeto', 'ano_referencia', 'empresa_razao_social', 'cnpj', 
                     'nome_projeto', 'descricao_projeto', 'tipo_projeto', 'natureza',
                     'elemento_tecnologico', 'desafio_tecnologico', 'metodologia',
                     'setor_analise', 'ciclo_maior_1_ano']

colunas_faltando = [col for col in colunas_esperadas if col not in df.columns]
if colunas_faltando:
    print(f"\n⚠️ Colunas faltando: {colunas_faltando}")
    print("Colunas disponíveis:")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i:3d}. {col}")
else:
    print("✓ Todas as colunas esperadas encontradas")

# Continuar com a análise se temos as colunas principais
if all(col in df.columns for col in ['cnpj', 'ano_referencia', 'nome_projeto']):
    print(f"\nEmpresas únicas: {df['cnpj'].nunique():,}")
    print(f"Anos disponíveis: {sorted(df['ano_referencia'].dropna().unique())}")
    
    # Análise de projetos multianuais
    if 'ciclo_maior_1_ano' in df.columns:
        df['projeto_multianual'] = df['ciclo_maior_1_ano'] == 'Sim'
        print(f"\nProjetos multianuais: {df['projeto_multianual'].sum():,} ({df['projeto_multianual'].mean()*100:.1f}%)")
    else:
        print("\n⚠️ Coluna 'ciclo_maior_1_ano' não encontrada")
    
    # Criar identificador único para rastrear projetos entre anos
    df['projeto_chave'] = df['cnpj'].astype(str) + '_' + df['nome_projeto'].fillna('').astype(str).str[:50]
    
    # Distribuição por setor
    if 'setor_analise' in df.columns:
        print("\nDistribuição por setor:")
        print(df['setor_analise'].value_counts())
    else:
        print("\n⚠️ Coluna 'setor_analise' não encontrada")
    
    # Verificar anos com dados
    print("\nProjetos por ano:")
    print(df['ano_referencia'].value_counts().sort_index())
else:
    print("\n❌ Colunas essenciais não encontradas. Verifique o arquivo CSV.")

# Mostrar informações sobre o dataset
print("\n" + "="*80)
print("INFORMAÇÕES DO DATASET")
print("="*80)
print(df.info())

# Verificar primeiras linhas
print("\nPrimeiras 3 linhas do dataset:")
print(df.head(3))

Tentando carregar o dataset...
✓ Dados carregados

DADOS CARREGADOS COM SUCESSO
Total de registros: 74,502
Total de colunas: 26

APLICANDO FILTRO CRÍTICO - REMOVENDO 2023
Registros por ano ANTES do filtro:
ano_referencia
2018    10876
2019    12168
2020    11660
2021    13198
2022    13786
2023    12814
Name: count, dtype: int64

✓ Removidos 12,814 registros de 2023 (17.2%)
✓ Dataset filtrado: 61,688 registros (2018-2022)

Registros por ano APÓS filtro:
ano_referencia
2018    10876
2019    12168
2020    11660
2021    13198
2022    13786
Name: count, dtype: int64
✓ Todas as colunas esperadas encontradas

Empresas únicas: 4,885
Anos disponíveis: [np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022)]

Projetos multianuais: 46,238 (75.0%)

Distribuição por setor:
setor_analise
TIC                          16272
Química e Farmácia           11708
Mecânica e Transporte         7990
Agroindústria e Alimentos     7499
Transversal                   7090
Eletroeletrônic

In [2]:
# ============================================================
# CHUNK 2: ANÁLISE DEFINITIVA COM GRANITE - THRESHOLD 0.85
# ============================================================

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("ANÁLISE DE SIMILARIDADE COM GRANITE IBM - THRESHOLD 0.85")
print("="*80)

# Filtrar dados
print("\nFiltrando dados de Eletroeletrônica - 2021 - Processo...")
filtro = (
    (df['setor_analise'] == 'Eletroeletrônica') & 
    (df['ano_referencia'] == 2021) & 
    (df['natureza'] == 'Processo')
)

eletr_2021_proc = df[filtro].copy()
print(f"✓ Projetos filtrados: {len(eletr_2021_proc)}")

# ============================================================
# PREPARAÇÃO DOS TEXTOS
# ============================================================

print("\n" + "="*80)
print("PREPARAÇÃO DOS DADOS")
print("="*80)

# Campos de texto principais
campos_texto = ['nome_projeto', 'elemento_tecnologico', 'desafio_tecnologico', 
                'metodologia', 'descricao_projeto']

# Limpar valores nulos
for campo in campos_texto:
    if campo in eletr_2021_proc.columns:
        eletr_2021_proc[campo] = eletr_2021_proc[campo].fillna('').astype(str)

# Criar texto completo sem truncamento
def criar_texto_completo(row):
    """Concatena todos os campos disponíveis"""
    partes = []
    for campo in campos_texto:
        if campo in row and row[campo] and str(row[campo]).strip():
            partes.append(str(row[campo]).strip())
    return " ".join(partes)

eletr_2021_proc['texto_completo'] = eletr_2021_proc.apply(criar_texto_completo, axis=1)

# Estatísticas dos textos
comprimentos = eletr_2021_proc['texto_completo'].str.len()
print(f"\nEstatísticas dos textos:")
print(f"  - Total de projetos: {len(eletr_2021_proc)}")
print(f"  - Comprimento médio: {comprimentos.mean():.0f} caracteres")
print(f"  - Comprimento mínimo: {comprimentos.min():.0f} caracteres")
print(f"  - Comprimento máximo: {comprimentos.max():.0f} caracteres")

# Verificar e remover textos vazios
textos_vazios = (comprimentos == 0).sum()
if textos_vazios > 0:
    print(f"  ⚠️ Removendo {textos_vazios} projetos sem texto")
    eletr_2021_proc = eletr_2021_proc[comprimentos > 0].copy()

# ============================================================
# GERAR EMBEDDINGS COM GRANITE
# ============================================================

print("\n" + "="*80)
print("GERANDO EMBEDDINGS COM GRANITE IBM")
print("="*80)

# Carregar modelo Granite
print("Carregando modelo ibm-granite/granite-embedding-278m-multilingual...")
try:
    model = SentenceTransformer('ibm-granite/granite-embedding-278m-multilingual')
    print("✓ Modelo Granite carregado com sucesso")
except:
    print("⚠️ Usando modelo alternativo multilíngue...")
    model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
    print("✓ Modelo alternativo carregado")

# Preparar textos
textos = eletr_2021_proc['texto_completo'].tolist()
ids_projetos = eletr_2021_proc['id_projeto'].tolist() if 'id_projeto' in eletr_2021_proc.columns else eletr_2021_proc.index.tolist()
nomes_projetos = eletr_2021_proc['nome_projeto'].tolist()

print(f"\nGerando embeddings para {len(textos)} projetos...")

# Gerar embeddings
embeddings = model.encode(
    textos,
    batch_size=16,
    show_progress_bar=True,
    normalize_embeddings=True,
    convert_to_numpy=True
)

print(f"✓ Embeddings gerados: shape {embeddings.shape}")

# ============================================================
# CALCULAR MATRIZ DE SIMILARIDADE
# ============================================================

print("\n" + "="*80)
print("CALCULANDO SIMILARIDADES")
print("="*80)

# Calcular matriz de similaridade
print("Calculando matriz de similaridade...")
similarity_matrix = cosine_similarity(embeddings)

# Estatísticas gerais
upper_triangle = np.triu(similarity_matrix, k=1)
all_similarities = upper_triangle[upper_triangle > 0]

print(f"\n✓ Matriz calculada: {similarity_matrix.shape}")
print(f"  - Total de comparações únicas: {len(all_similarities):,}")
print(f"\nEstatísticas de similaridade:")
print(f"  - Média: {all_similarities.mean():.4f}")
print(f"  - Desvio padrão: {all_similarities.std():.4f}")
print(f"  - Mínima: {all_similarities.min():.4f}")
print(f"  - Máxima: {all_similarities.max():.4f}")
print(f"  - Mediana: {np.median(all_similarities):.4f}")

# Percentis importantes
percentis = [75, 80, 85, 90, 95, 99]
print(f"\nPercentis:")
for p in percentis:
    valor = np.percentile(all_similarities, p)
    print(f"  - {p}%: {valor:.4f}")

# ============================================================
# IDENTIFICAR PARES SIMILARES (>= 0.85)
# ============================================================

print("\n" + "="*80)
print("IDENTIFICANDO PARES SIMILARES (THRESHOLD >= 0.85)")
print("="*80)

threshold = 0.85
pares_similares = []

# Iterar sobre triangular superior para evitar duplicatas
print("Procurando pares com similaridade >= 0.85...")
for i in range(len(ids_projetos)):
    for j in range(i + 1, len(ids_projetos)):
        similaridade = similarity_matrix[i, j]
        
        if similaridade >= threshold:
            # Coletar informações adicionais se disponíveis
            registro = {
                'IdProjeto_1': ids_projetos[i],
                'NomeProjeto_1': nomes_projetos[i],
                'IdProjeto_2': ids_projetos[j],
                'NomeProjeto_2': nomes_projetos[j],
                'Similaridade_score': float(similaridade)
            }
            
            # Adicionar informações de empresa se disponíveis
            if 'empresa_razao_social' in eletr_2021_proc.columns:
                registro['Empresa_1'] = eletr_2021_proc.iloc[i]['empresa_razao_social']
                registro['Empresa_2'] = eletr_2021_proc.iloc[j]['empresa_razao_social']
                registro['Mesma_Empresa'] = registro['Empresa_1'] == registro['Empresa_2']
            
            if 'cnpj' in eletr_2021_proc.columns:
                registro['CNPJ_1'] = eletr_2021_proc.iloc[i]['cnpj']
                registro['CNPJ_2'] = eletr_2021_proc.iloc[j]['cnpj']
            
            pares_similares.append(registro)

# Criar DataFrame com pares similares
eletr_2021_proc_similar = pd.DataFrame(pares_similares)

if len(eletr_2021_proc_similar) > 0:
    # Ordenar por similaridade
    eletr_2021_proc_similar = eletr_2021_proc_similar.sort_values('Similaridade_score', ascending=False)
    
    print(f"\n✓ Pares encontrados: {len(eletr_2021_proc_similar)}")
    
    # Estatísticas dos pares
    print(f"\nEstatísticas dos pares similares:")
    print(f"  - Similaridade máxima: {eletr_2021_proc_similar['Similaridade_score'].max():.4f}")
    print(f"  - Similaridade média: {eletr_2021_proc_similar['Similaridade_score'].mean():.4f}")
    print(f"  - Similaridade mínima: {eletr_2021_proc_similar['Similaridade_score'].min():.4f}")
    
    # Análise por faixas
    print(f"\nDistribuição por faixas:")
    faixas = [(0.98, 1.00), (0.95, 0.98), (0.90, 0.95), (0.85, 0.90)]
    for min_val, max_val in faixas:
        if max_val == 1.00:
            count = (eletr_2021_proc_similar['Similaridade_score'] >= min_val).sum()
        else:
            count = ((eletr_2021_proc_similar['Similaridade_score'] >= min_val) & 
                    (eletr_2021_proc_similar['Similaridade_score'] < max_val)).sum()
        pct = count / len(eletr_2021_proc_similar) * 100 if len(eletr_2021_proc_similar) > 0 else 0
        print(f"  - [{min_val:.2f}, {max_val:.2f}): {count} pares ({pct:.1f}%)")
    
    # Top 10 pares mais similares
    print(f"\nTop 10 pares mais similares:")
    print("-" * 80)
    for idx, row in eletr_2021_proc_similar.head(10).iterrows():
        print(f"\nSimilaridade: {row['Similaridade_score']:.4f}")
        print(f"  Projeto 1: [{row['IdProjeto_1']}] {row['NomeProjeto_1'][:50]}...")
        print(f"  Projeto 2: [{row['IdProjeto_2']}] {row['NomeProjeto_2'][:50]}...")
        if 'Mesma_Empresa' in row:
            tipo = "MESMA EMPRESA" if row['Mesma_Empresa'] else "EMPRESAS DIFERENTES"
            print(f"  Tipo: {tipo}")
    
    # Análise por empresa se disponível
    if 'Mesma_Empresa' in eletr_2021_proc_similar.columns:
        mesma_empresa = eletr_2021_proc_similar['Mesma_Empresa'].sum()
        diferentes = len(eletr_2021_proc_similar) - mesma_empresa
        print(f"\n" + "="*80)
        print("ANÁLISE POR TIPO DE PAR")
        print("="*80)
        print(f"  - Pares da mesma empresa: {mesma_empresa} ({mesma_empresa/len(eletr_2021_proc_similar)*100:.1f}%)")
        print(f"  - Pares entre empresas diferentes: {diferentes} ({diferentes/len(eletr_2021_proc_similar)*100:.1f}%)")
        
        # Empresas com mais pares similares
        if 'Empresa_1' in eletr_2021_proc_similar.columns:
            print(f"\nEmpresas com mais projetos similares:")
            todas_empresas = pd.concat([
                eletr_2021_proc_similar['Empresa_1'],
                eletr_2021_proc_similar['Empresa_2']
            ])
            top_empresas = todas_empresas.value_counts().head(5)
            for empresa, count in top_empresas.items():
                print(f"  - {empresa[:50]}: {count} aparições em pares similares")
else:
    print(f"\n⚠️ Nenhum par encontrado com similaridade >= {threshold}")

# ============================================================
# SALVAR RESULTADOS EM CSV
# ============================================================

print("\n" + "="*80)
print("SALVANDO RESULTADOS")
print("="*80)

# Remover coluna temporária antes de salvar
if 'texto_completo' in eletr_2021_proc.columns:
    eletr_2021_proc_export = eletr_2021_proc.drop(columns=['texto_completo'])
else:
    eletr_2021_proc_export = eletr_2021_proc.copy()

# Salvar dataset filtrado
arquivo_projetos = 'eletr_2021_proc.csv'
eletr_2021_proc_export.to_csv(arquivo_projetos, index=False, encoding='utf-8-sig', sep=';')
print(f"✓ Dataset filtrado salvo: {arquivo_projetos}")
print(f"  - Total de projetos: {len(eletr_2021_proc_export)}")
print(f"  - Total de colunas: {eletr_2021_proc_export.shape[1]}")

# Salvar pares similares
if len(eletr_2021_proc_similar) > 0:
    # Selecionar colunas essenciais para o CSV
    colunas_essenciais = ['IdProjeto_1', 'NomeProjeto_1', 'IdProjeto_2', 'NomeProjeto_2', 'Similaridade_score']
    colunas_extras = ['Empresa_1', 'Empresa_2', 'Mesma_Empresa', 'CNPJ_1', 'CNPJ_2']
    
    # Adicionar colunas extras se existirem
    colunas_export = colunas_essenciais + [col for col in colunas_extras if col in eletr_2021_proc_similar.columns]
    eletr_2021_proc_similar_export = eletr_2021_proc_similar[colunas_export]
    
    arquivo_similares = 'eletr_2021_proc_similar.csv'
    eletr_2021_proc_similar_export.to_csv(arquivo_similares, index=False, encoding='utf-8-sig', sep=';')
    print(f"\n✓ Pares similares salvos: {arquivo_similares}")
    print(f"  - Total de pares: {len(eletr_2021_proc_similar_export)}")
    print(f"  - Colunas incluídas: {', '.join(colunas_export)}")
else:
    # Criar arquivo vazio com estrutura básica
    df_vazio = pd.DataFrame(columns=['IdProjeto_1', 'NomeProjeto_1', 'IdProjeto_2', 'NomeProjeto_2', 'Similaridade_score'])
    df_vazio.to_csv('eletr_2021_proc_similar.csv', index=False, encoding='utf-8-sig', sep=';')
    print(f"\n✓ Arquivo de pares similares criado (vazio - nenhum par acima do threshold)")

# ============================================================
# RESUMO FINAL
# ============================================================

print("\n" + "="*80)
print("ANÁLISE CONCLUÍDA COM SUCESSO")
print("="*80)

print(f"\n📊 RESUMO EXECUTIVO:")
print(f"  • Modelo utilizado: Granite IBM Multilingual")
print(f"  • Projetos analisados: {len(eletr_2021_proc)}")
print(f"  • Threshold aplicado: {threshold}")
print(f"  • Pares similares identificados: {len(eletr_2021_proc_similar)}")

if len(eletr_2021_proc_similar) > 0:
    taxa = len(eletr_2021_proc_similar) / (len(eletr_2021_proc) * (len(eletr_2021_proc) - 1) / 2) * 100
    print(f"  • Taxa de pares similares: {taxa:.3f}%")

print(f"\n📁 Arquivos gerados:")
print(f"  1. eletr_2021_proc.csv - Base completa filtrada")
print(f"  2. eletr_2021_proc_similar.csv - Pares com similaridade >= 0.85")

2025-11-19 11:07:04.020789: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ANÁLISE DE SIMILARIDADE COM GRANITE IBM - THRESHOLD 0.85

Filtrando dados de Eletroeletrônica - 2021 - Processo...
✓ Projetos filtrados: 366

PREPARAÇÃO DOS DADOS

Estatísticas dos textos:
  - Total de projetos: 366
  - Comprimento médio: 8058 caracteres
  - Comprimento mínimo: 1077 caracteres
  - Comprimento máximo: 16166 caracteres

GERANDO EMBEDDINGS COM GRANITE IBM
Carregando modelo ibm-granite/granite-embedding-278m-multilingual...
✓ Modelo Granite carregado com sucesso

Gerando embeddings para 366 projetos...


Batches:   0%|          | 0/23 [00:00<?, ?it/s]

✓ Embeddings gerados: shape (366, 768)

CALCULANDO SIMILARIDADES
Calculando matriz de similaridade...

✓ Matriz calculada: (366, 366)
  - Total de comparações únicas: 66,795

Estatísticas de similaridade:
  - Média: 0.6096
  - Desvio padrão: 0.0578
  - Mínima: 0.4090
  - Máxima: 1.0000
  - Mediana: 0.6069

Percentis:
  - 75%: 0.6459
  - 80%: 0.6556
  - 85%: 0.6678
  - 90%: 0.6831
  - 95%: 0.7071
  - 99%: 0.7563

IDENTIFICANDO PARES SIMILARES (THRESHOLD >= 0.85)
Procurando pares com similaridade >= 0.85...

✓ Pares encontrados: 70

Estatísticas dos pares similares:
  - Similaridade máxima: 1.0000
  - Similaridade média: 0.9478
  - Similaridade mínima: 0.8500

Distribuição por faixas:
  - [0.98, 1.00): 33 pares (47.1%)
  - [0.95, 0.98): 10 pares (14.3%)
  - [0.90, 0.95): 8 pares (11.4%)
  - [0.85, 0.90): 19 pares (27.1%)

Top 10 pares mais similares:
--------------------------------------------------------------------------------

Similaridade: 1.0000
  Projeto 1: [68931] Torre Móvel - D

In [5]:
# ============================================================
# CHUNK 3: VALIDAÇÃO ASSÍNCRONA COM LLM LOCAL (OLLAMA)
# VERSÃO COM PACOTE OLLAMA - PROCESSA TODOS OS PARES
# ============================================================

import pandas as pd
import numpy as np
import ollama
import asyncio
import json
import time
import warnings
from concurrent.futures import ThreadPoolExecutor
warnings.filterwarnings('ignore')

print("="*80)
print("VALIDAÇÃO ASSÍNCRONA DE SIMILARIDADE COM LLM LOCAL (OLLAMA)")
print("="*80)

# Carregar os dados
print("\nCarregando dados...")
eletr_2021_proc = pd.read_csv('eletr_2021_proc.csv', sep=';', encoding='utf-8-sig')
eletr_2021_proc_similar = pd.read_csv('eletr_2021_proc_similar.csv', sep=';', encoding='utf-8-sig')

print(f"✓ Projetos carregados: {len(eletr_2021_proc)}")
print(f"✓ Pares similares carregados: {len(eletr_2021_proc_similar)}")

# ============================================================
# CONFIGURAÇÃO DO OLLAMA
# ============================================================

print("\n" + "="*80)
print("CONFIGURAÇÃO DO OLLAMA")
print("="*80)

# Configuração simplificada - usa pacote ollama diretamente
MODEL = "qwen3:30b"  # Altere para o modelo que você tem disponível
# Modelos comuns: "llama2", "mistral", "qwen3:30b", "qwen3:32b", "gemma:2b"
BATCH_SIZE = 5  # Processar 5 requisições simultâneas

print(f"📦 Modelo configurado: {MODEL}")
print(f"⚙️  Batch size: {BATCH_SIZE}")

# Verificar se o modelo está disponível
try:
    modelos_disponiveis = [m['name'] for m in ollama.list()['models']]
    print(f"\n✓ Modelos disponíveis no Ollama:")
    for modelo in modelos_disponiveis:
        print(f"  - {modelo}")
    
    if MODEL not in modelos_disponiveis:
        print(f"\n⚠️ ATENÇÃO: Modelo '{MODEL}' não encontrado!")
        print(f"💡 Baixe o modelo com: ollama pull {MODEL}")
        print(f"💡 Ou altere a variável MODEL para um dos modelos acima")
except Exception as e:
    print(f"⚠️ Erro ao listar modelos: {e}")
    print("💡 Certifique-se que o Ollama está rodando: ollama serve")

# ============================================================
# DEFINIR CRITÉRIOS DE AVALIAÇÃO
# ============================================================

print("\n" + "="*80)
print("CRITÉRIOS DE INOVAÇÃO - LEI DO BEM")
print("="*80)

criterios_avaliacao = """
CRITÉRIOS DE AVALIAÇÃO DE INOVAÇÃO (Lei 11.196/2005):

1. **Novidade ou Aperfeiçoamento** - O projeto introduz algo novo ou melhora significativamente algo existente?
2. **Risco Tecnológico** - Existe incerteza quanto à viabilidade técnica?
3. **Incremento Tecnológico** - Há avanço no conhecimento ou capacitação tecnológica?
4. **Aplicação Industrial** - O resultado pode ser aplicado em processos produtivos?
5. **Esforço Tecnológico** - Demanda conhecimento técnico-científico especializado?

CLASSIFICAÇÃO DE SIMILARIDADE:
- IDÊNTICOS: Mesma solução técnica, mesmo problema, mesma abordagem
- MUITO SIMILARES: Problema similar, soluções comparáveis, mesmo nível de inovação
- SIMILARES: Mesma área tecnológica, complexidade equivalente
- DISTINTOS: Apesar de vocabulário similar, são projetos tecnicamente diferentes
"""

print(criterios_avaliacao)

# ============================================================
# FUNÇÃO PARA ANÁLISE VIA LLM (USANDO PACOTE OLLAMA)
# ============================================================

def analisar_par_projetos(projeto1_data, projeto2_data, empresa1, empresa2, par_id, id_proj1, id_proj2):
    """
    Analisa um par de projetos usando o LLM local via pacote ollama
    VERSÃO SIMPLIFICADA - Usa ollama.generate() diretamente
    """
    
    # Montar o prompt estruturado
    prompt = f"""
Você é um especialista em avaliação de projetos de P&D para a Lei do Bem (Lei 11.196/2005). 
Analise os dois projetos abaixo e determine se devem ser tratados com os mesmos critérios de avaliação.

{criterios_avaliacao}

**PROJETO 1** - Empresa: {empresa1}
Nome: {projeto1_data['nome_projeto']}
Elemento Tecnológico: {projeto1_data['elemento_tecnologico'][:1000]}
Desafio Tecnológico: {projeto1_data['desafio_tecnologico'][:1000]}
Metodologia: {projeto1_data['metodologia'][:800]}

**PROJETO 2** - Empresa: {empresa2}
Nome: {projeto2_data['nome_projeto']}
Elemento Tecnológico: {projeto2_data['elemento_tecnologico'][:1000]}
Desafio Tecnológico: {projeto2_data['desafio_tecnologico'][:1000]}
Metodologia: {projeto2_data['metodologia'][:800]}

RESPONDA APENAS EM FORMATO JSON PURO (sem markdown, sem ```):
{{
    "classificacao": "IDÊNTICOS|MUITO_SIMILARES|SIMILARES|DISTINTOS",
    "mesmo_tratamento": true/false,
    "justificativa": "explicação breve",
    "nivel_inovacao_proj1": 1-5,
    "nivel_inovacao_proj2": 1-5,
    "risco_tecnologico_similar": true/false,
    "area_tecnologica": "descrição da área",
    "alerta_duplicacao": true/false
}}
"""
    
    try:
        # Usar ollama.generate() - método síncrono mais simples
        response = ollama.generate(
            model=MODEL,
            prompt=prompt,
            options={
                'temperature': 0.1,
                'num_predict': 500,
            }
        )
        
        llm_response = response['response']
        
        # Tentar extrair JSON da resposta
        import re
        
        # Remover markdown se existir
        llm_response = llm_response.replace('```json', '').replace('```', '').strip()
        
        # Procurar por JSON
        json_match = re.search(r'\{.*\}', llm_response, re.DOTALL)
        
        if json_match:
            json_str = json_match.group()
            parsed_json = json.loads(json_str)
            parsed_json['par_id'] = par_id
            parsed_json['id_projeto_1'] = id_proj1
            parsed_json['id_projeto_2'] = id_proj2
            return parsed_json
        else:
            return {
                "erro": "JSON não encontrado na resposta do LLM", 
                "par_id": par_id,
                "id_projeto_1": id_proj1,
                "id_projeto_2": id_proj2,
                "resposta_raw": llm_response[:200]
            }
            
    except json.JSONDecodeError as e:
        return {
            "erro": f"Erro ao decodificar JSON: {str(e)}", 
            "par_id": par_id,
            "id_projeto_1": id_proj1,
            "id_projeto_2": id_proj2,
            "resposta_raw": llm_response[:200] if 'llm_response' in locals() else "N/A"
        }
    except Exception as e:
        return {
            "erro": f"Exceção: {str(e)}", 
            "par_id": par_id,
            "id_projeto_1": id_proj1,
            "id_projeto_2": id_proj2
        }

# ============================================================
# PROCESSAR LOTES DE PARES COM THREADING
# ============================================================

def processar_lote(lote_pares, eletr_2021_proc, executor):
    """
    Processa um lote de pares usando ThreadPoolExecutor
    """
    tarefas = []
    
    for idx, row in lote_pares.iterrows():
        try:
            # Buscar dados completos dos projetos
            id_proj1 = row['IdProjeto_1']
            id_proj2 = row['IdProjeto_2']
            
            if 'id_projeto' in eletr_2021_proc.index.names:
                proj1_data = eletr_2021_proc.loc[id_proj1]
                proj2_data = eletr_2021_proc.loc[id_proj2]
            else:
                proj1_data = eletr_2021_proc[eletr_2021_proc['nome_projeto'] == row['NomeProjeto_1']].iloc[0]
                proj2_data = eletr_2021_proc[eletr_2021_proc['nome_projeto'] == row['NomeProjeto_2']].iloc[0]
            
            empresa1 = row.get('Empresa_1', 'Empresa 1')[:50] if 'Empresa_1' in row else 'Empresa 1'
            empresa2 = row.get('Empresa_2', 'Empresa 2')[:50] if 'Empresa_2' in row else 'Empresa 2'
            
            # Criar tarefa para o executor
            future = executor.submit(
                analisar_par_projetos,
                proj1_data, proj2_data, 
                empresa1, empresa2, idx, id_proj1, id_proj2
            )
            tarefas.append((idx, row, id_proj1, id_proj2, future))
            
        except Exception as e:
            print(f"Erro ao preparar par {idx}: {e}")
    
    # Coletar resultados
    resultados = []
    for idx, row, id_proj1, id_proj2, future in tarefas:
        resultado = future.result()
        
        # Adicionar informações do par
        resultado.update({
            'nome_projeto_1': row['NomeProjeto_1'],
            'nome_projeto_2': row['NomeProjeto_2'],
            'similaridade_coseno': row['Similaridade_score'],
            'mesma_empresa': row.get('Mesma_Empresa', False)
        })
        resultados.append(resultado)
    
    return resultados

def processar_todos_pares():
    """
    Função principal para processar todos os pares em lotes
    """
    print("\n" + "="*80)
    print("INICIANDO ANÁLISE DOS PARES")
    print("="*80)
    
    # Preparar índice para busca
    if 'id_projeto' in eletr_2021_proc.columns:
        eletr_2021_proc.set_index('id_projeto', inplace=True)
    
    # Processar todos os pares
    pares_para_analisar = eletr_2021_proc_similar
    total_pares = len(pares_para_analisar)
    
    print(f"\n📊 Total de pares para analisar: {total_pares}")
    print(f"⚙️  Processando em lotes de {BATCH_SIZE}...")
    
    # Dividir em lotes
    n_lotes = (total_pares + BATCH_SIZE - 1) // BATCH_SIZE
    todos_resultados = []
    
    # Usar ThreadPoolExecutor para paralelização
    with ThreadPoolExecutor(max_workers=BATCH_SIZE) as executor:
        # Processar cada lote
        for i in range(n_lotes):
            inicio = i * BATCH_SIZE
            fim = min(inicio + BATCH_SIZE, total_pares)
            lote = pares_para_analisar.iloc[inicio:fim]
            
            print("="*80)
            print(f"Processando lote {i+1}/{n_lotes} (pares {inicio+1}-{fim})...")
            print("="*80)
            
            # Processar lote
            resultados_lote = processar_lote(lote, eletr_2021_proc, executor)
            todos_resultados.extend(resultados_lote)
            
            # Output detalhado com IDs dos projetos
            for res in resultados_lote:
                id1 = res.get('id_projeto_1', 'N/A')
                id2 = res.get('id_projeto_2', 'N/A')
                
                if 'erro' not in res:
                    classif = res.get('classificacao', 'N/A')
                    print(f"  ✓ Par Id {id1} e Id {id2}: {classif}")
                else:
                    erro = res.get('erro', 'Erro desconhecido')
                    print(f"  ❌ Par Id {id1} e Id {id2}: ERRO - {erro[:50]}")
            
            # Estatísticas parciais a cada 10 lotes
            if (i + 1) % 10 == 0 or (i + 1) == n_lotes:
                processados = len(todos_resultados)
                com_sucesso = sum(1 for r in todos_resultados if 'erro' not in r)
                com_erro = processados - com_sucesso
                
                print("\n" + "-"*80)
                print(f"📊 PROGRESSO: {processados}/{total_pares} pares processados")
                print(f"   ✓ Sucessos: {com_sucesso} ({com_sucesso/processados*100:.1f}%)")
                print(f"   ❌ Erros: {com_erro} ({com_erro/processados*100:.1f}%)")
                print("-"*80 + "\n")
            
            # Pequena pausa entre lotes
            if i < n_lotes - 1:
                time.sleep(1)
    
    return todos_resultados

# ============================================================
# EXECUTAR ANÁLISE
# ============================================================

print("\nIniciando processamento...")
inicio_total = time.time()

resultados_analise = processar_todos_pares()

tempo_total = time.time() - inicio_total

# ============================================================
# ANÁLISE DOS RESULTADOS
# ============================================================

print("\n" + "="*80)
print("ANÁLISE DOS RESULTADOS DA VALIDAÇÃO")
print("="*80)

if resultados_analise:
    # Converter para DataFrame
    df_resultados = pd.DataFrame(resultados_analise)
    
    # Estatísticas gerais
    total_processado = len(df_resultados)
    com_erro = (df_resultados['erro'].notna()).sum() if 'erro' in df_resultados.columns else 0
    sem_erro = total_processado - com_erro
    
    print(f"\n📊 ESTATÍSTICAS GERAIS:")
    print(f"  Total processado: {total_processado}")
    print(f"  Processamento bem-sucedido: {sem_erro} ({sem_erro/total_processado*100:.1f}%)")
    print(f"  Erros: {com_erro} ({com_erro/total_processado*100:.1f}%)")
    print(f"  Tempo total: {tempo_total/60:.1f} minutos")
    print(f"  Tempo médio por par: {tempo_total/total_processado:.2f} segundos")
    
    # Análise se tivermos resultados válidos
    if 'classificacao' in df_resultados.columns:
        # Remover linhas com erro
        df_validos = df_resultados[~df_resultados['classificacao'].isna()]
        
        print(f"\n✓ Resultados válidos para análise: {len(df_validos)}")
        
        # Estatísticas de classificação
        print("\n" + "="*80)
        print("DISTRIBUIÇÃO DE CLASSIFICAÇÕES")
        print("="*80)
        classificacao_counts = df_validos['classificacao'].value_counts()
        for classif, count in classificacao_counts.items():
            print(f"  {classif:20s}: {count:4d} ({count/len(df_validos)*100:5.1f}%)")
        
        # Análise de tratamento similar
        if 'mesmo_tratamento' in df_validos.columns:
            mesmo_trat = df_validos['mesmo_tratamento'].sum()
            print(f"\n📋 Projetos que devem ter MESMO tratamento: {mesmo_trat}/{len(df_validos)} ({mesmo_trat/len(df_validos)*100:.1f}%)")
        
        # Alertas de duplicação
        if 'alerta_duplicacao' in df_validos.columns:
            alertas = df_validos['alerta_duplicacao'].sum()
            print(f"⚠️  Alertas de possível duplicação: {alertas}")
        
        # Top 10 pares críticos
        print("\n" + "="*80)
        print("TOP 10 PARES MAIS CRÍTICOS")
        print("="*80)
        criticos = df_validos[
            (df_validos.get('classificacao', '') == 'IDÊNTICOS') | 
            (df_validos.get('alerta_duplicacao', False) == True)
        ].sort_values('similaridade_coseno', ascending=False)
        
        for i, (_, row) in enumerate(criticos.head(10).iterrows(), 1):
            print(f"\n{i}. Par: Id {row['id_projeto_1']} ↔ Id {row['id_projeto_2']}")
            print(f"   Projeto 1: {row['nome_projeto_1'][:60]}...")
            print(f"   Projeto 2: {row['nome_projeto_2'][:60]}...")
            print(f"   Classificação: {row.get('classificacao', 'N/A')}")
            print(f"   Similaridade: {row['similaridade_coseno']:.3f}")
            if 'justificativa' in row:
                print(f"   Justificativa: {row['justificativa'][:100]}...")
    
    # ============================================================
    # SALVAR RESULTADOS
    # ============================================================
    
    print("\n" + "="*80)
    print("SALVANDO RESULTADOS")
    print("="*80)
    
    arquivo_validacao = 'validacao_llm_pares_similares.csv'
    df_resultados.to_csv(arquivo_validacao, index=False, encoding='utf-8-sig', sep=';')
    print(f"✓ Resultados salvos em: {arquivo_validacao}")
    
else:
    print("❌ Nenhum resultado obtido.")

print("\n" + "="*80)
print("ANÁLISE CONCLUÍDA")
print("="*80)

VALIDAÇÃO ASSÍNCRONA DE SIMILARIDADE COM LLM LOCAL (OLLAMA)

Carregando dados...
✓ Projetos carregados: 366
✓ Pares similares carregados: 70

CONFIGURAÇÃO DO OLLAMA
📦 Modelo configurado: qwen3:30b
⚙️  Batch size: 5
⚠️ Erro ao listar modelos: 'name'
💡 Certifique-se que o Ollama está rodando: ollama serve

CRITÉRIOS DE INOVAÇÃO - LEI DO BEM

CRITÉRIOS DE AVALIAÇÃO DE INOVAÇÃO (Lei 11.196/2005):

1. **Novidade ou Aperfeiçoamento** - O projeto introduz algo novo ou melhora significativamente algo existente?
2. **Risco Tecnológico** - Existe incerteza quanto à viabilidade técnica?
3. **Incremento Tecnológico** - Há avanço no conhecimento ou capacitação tecnológica?
4. **Aplicação Industrial** - O resultado pode ser aplicado em processos produtivos?
5. **Esforço Tecnológico** - Demanda conhecimento técnico-científico especializado?

CLASSIFICAÇÃO DE SIMILARIDADE:
- IDÊNTICOS: Mesma solução técnica, mesmo problema, mesma abordagem
- MUITO SIMILARES: Problema similar, soluções comparáveis, mes